https://github.com/JoMingyu/google-play-scraper

In [1]:
!pip install -qq google-play-scraper

In [8]:
!pip install Pygments


In [19]:
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import TerminalFormatter

from google_play_scraper import app
from google_play_scraper import Sort, reviews   ## from https://github.com/JoMingyu/google-play-scraper document

%matplotlib inline
%config InlineBackend.print_figure_format='retina'

sns.set(style = 'whitegrid', palette='muted', font_scale=1.2)

App Detail https://github.com/JoMingyu/google-play-scraper#app-detail

from google_play_scraper import app

result = app(
    'com.nianticlabs.pokemongo',
    lang='en', # defaults to 'en'
    country='us' # defaults to 'us'
)


In [17]:
app_packages = [
    'com.asda.android',
    'com.morrisons.atm.mobile.android',
    'com.sainsburys.gol',
    'com.tesco.grocery.view'

]

In [20]:
app_infos = []

for ap in tqdm(app_packages):
  info = app(ap, lang = 'en', country='uk')
  del info['comments']
  app_infos.append(info)


100%|██████████| 4/4 [00:01<00:00,  3.98it/s]


In [22]:
def print_json(json_object):
  json_str = json.dumps(
    json_object,
    indent=2,
    sort_keys=True,
    default=str
  )
  print(highlight(json_str, JsonLexer(), TerminalFormatter()))

In [23]:
print_json(app_infos[0])

{
  "adSupported": false,
  "appId": "com.asda.android",
  "categories": [
    {
      "id": "SHOPPING",
      "name": "Shopping"
    }
  ],
  "containsAds": false,
  "contentRating": "PEGI 3",
  "contentRatingDescription": null,
  "currency": "GBP",
  "description": "ASDA\u2019s FREE\u00b1 grocery shopping app lets you shop for groceries on the go. Get all the amazing in-store offers & easily book a delivery or use our Click & Collect service.\r\n \r\nGrocery shopping made easy   \r\nShop your way: Available on all devices on the move. \r\nScan items at home: Running out of something? Build your shopping list quickly and easily.  \r\nCreate shopping lists: Organise your shop to save time when you next shop using the FREE\u00b1 Asda App.  \r\nShopped in-store before? : Transfer your favourite items on your receipt into your online basket. Clever!\r\nFind your nearest store: By turning on-location services, you can locate your nearest store wherever you are.  \r\n \r\nWith the ASDA groc

store the app information for later by converting the JSON objects into a Pandas dataframe and saving the result into a CSV file

In [29]:
app_infos_df = pd.DataFrame(app_infos)

In [31]:
app_infos_df.head()

,title,description,descriptionHTML,summary,installs,minInstalls,realInstalls,score,ratings,reviews,...,videoImage,contentRating,contentRatingDescription,adSupported,containsAds,released,updated,version,appId,url
0,ASDA,ASDA’s FREE± grocery shopping app lets you sho...,ASDA’s FREE± grocery shopping app lets you sho...,"Book a slot, shop your favourites, & order gro...","5,000,000+",5000000,5974394,4.505665,97162,41146,...,None,PEGI 3,None,False,False,"Apr 10, 2012",1688032480,23.02,com.asda.android,https://play.google.com/store/apps/details?id=...
1,Morrisons Groceries,"Food, drink and more to your door…\r\n\r\nNow ...","Food, drink and more to your door…<br><br>Now ...",Morrisons.com - from market street to your street,"500,000+",500000,970052,4.439716,8463,1738,...,None,PEGI 3,None,False,False,"Mar 20, 2019",1689365353,v0.318.1 morrisonsRelease,com.morrisons.atm.mobile.android,https://play.google.com/store/apps/details?id=...
2,Sainsbury's Groceries,You can now do your groceries shopping whereve...,You can now do your groceries shopping whereve...,Do your Sainsbury's shop on your phone and we’...,"1,000,000+",1000000,1754849,4.332637,14395,6576,...,None,PEGI 3,None,False,False,"May 26, 2016",1688635813,3.9.0,com.sainsburys.gol,https://play.google.com/store/apps/details?id=...
3,Tesco Grocery & Clubcard,Get more power in your pocket with the new Tes...,Get more power in your pocket with the new Tes...,"Shop groceries, use Clubcard","10,000,000+",10000000,11165973,4.082919,137665,62159,...,None,PEGI 3,None,True,True,"Jun 8, 2011",1689078023,19.52.2,com.tesco.grocery.view,https://play.google.com/store/apps/details?id=...


In [32]:
app_infos_df.to_csv('apps_info.csv', index=None, header=True)

Scraping App Reviews

In [43]:
app_reviews = []
for ap in tqdm(app_packages):
  for score in list(range(1, 6)):
    for sort_order in [Sort.MOST_RELEVANT, Sort.NEWEST]:
      rvs, _ = reviews(
        ap,
        lang='en',
        country='us',
        sort=sort_order,
        count= 200 if score == 3 else 100,
        filter_score_with=score
      )
      for r in rvs:
        r['sortOrder'] = 'most_relevant' if sort_order == Sort.MOST_RELEVANT else 'newest'
        r['appId'] = ap
      app_reviews.extend(rvs)

100%|██████████| 4/4 [00:17<00:00,  4.29s/it]


In [44]:
print_json(app_reviews[0])

{
  "appId": "com.asda.android",
  "appVersion": "23.01.03",
  "at": "2023-06-24 15:27:25",
  "content": "I dont like the latest update. I've used the Asda app for years. I used to be able to click on individual items in previous orders and add them to my shop. Now you can't do that and if you try to add an entire previous trolley, it wipes what is currently in it! Each item you add now has to be searched for individually. More time consuming and not helpful anymore!",
  "repliedAt": null,
  "replyContent": null,
  "reviewCreatedVersion": "23.01.03",
  "reviewId": "1752d5cc-e67c-40e7-bf38-cda11b5630e3",
  "score": 1,
  "sortOrder": "most_relevant",
  "thumbsUpCount": 111,
  "userImage": "https://play-lh.googleusercontent.com/a/AAcHTtddef8mS7D8BLsKbyF5QYlwbhDnKU4aWWZ1IVSbQzaKjg=mo",
  "userName": "Nicky"
}



In [45]:
len(app_reviews)

4632

In [46]:
app_reviews_df = pd.DataFrame(app_reviews)

In [48]:
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,sortOrder,appId
0,1752d5cc-e67c-40e7-bf38-cda11b5630e3,Nicky,https://play-lh.googleusercontent.com/a/AAcHTt...,I dont like the latest update. I've used the A...,1,111,23.01.03,2023-06-24 15:27:25,None,NaT,23.01.03,most_relevant,com.asda.android
1,832dd7f7-a914-4c02-ad42-b437bfc677d5,Archna Patel,https://play-lh.googleusercontent.com/a/AAcHTt...,"Sneaky asda, dont share offer ends dates, but ...",1,58,23.01.03,2023-05-07 06:19:28,None,NaT,23.01.03,most_relevant,com.asda.android
2,43a56d6c-9143-4bd7-a43f-7e82c64141e1,Ruth L2911,https://play-lh.googleusercontent.com/a/AAcHTt...,Worst App...EVER!! It freezes every single tim...,1,1,23.01.03,2023-05-23 00:01:14,None,NaT,23.01.03,most_relevant,com.asda.android
3,f10324c8-64ae-445c-b819-f7786b03f19d,scouseraph,https://play-lh.googleusercontent.com/a/AAcHTt...,I would love this app to include a feature whe...,1,1,23.01.03,2023-06-22 06:26:58,None,NaT,23.01.03,most_relevant,com.asda.android
4,2bed7537-026e-43b9-86d2-fc6150c757d4,Thorfinn,https://play-lh.googleusercontent.com/a-/AD_cM...,It's bloody awful. Sometimes the app freezes u...,1,0,23.01.03,2023-07-03 12:03:05,None,NaT,23.01.03,most_relevant,com.asda.android


In [49]:
app_reviews_df.to_csv('reviews.csv', index=None, header=True)